# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow.keras as keras

# Dataset Path Setting
make sure the paths are correct.


In [ ]:
class Datasets:
    BASE = "/media/sda1/dataset/NAB/data"
    DATA = "realAdExchange/"
    CSV1 = "exchange-2_cpc_results.csv"

    MFPT_BASE = "/media/sda1/dataset/MFPT/MFPT Fault Data Sets"
    MFPT_DATA = "1 - Three Baseline Conditions"
    MFPT_MAT = "baseline_1.mat"

# Data Preprocessing
Preprocessing datas before the training
Simply change dataset shape in this example.

In [ ]:
class Preprocessing:

    @staticmethod
    def create_dataset(data, window_size):
        
        dataset = np.empty((int(data.size/window_size), window_size, 1))
        j = 0
        for i in range(int(data.size/window_size)):
            dataset[j] = data[i:i+window_size]
            j = j + 1

        print(f"Dataset shape changed {data.shape} -> {dataset.shape}.")
        print(f"Truncated {data.size - dataset.size} values ")
        
        return dataset

# Configuring Model

In [ ]:
class Model:
    #https://machinelearningmastery.com/lstm-autoencoders/
    def __init__(self, x_train):
        self.x_train = x_train
    
    def lstm_autoencoder(self, input_size, epochs):
        output_size = input_size
        model = keras.models.Sequential([
            keras.layers.RNN(keras.layers.LSTMCell(128), input_shape=[input_size, 1]),
            keras.layers.RepeatVector(output_size),
            keras.layers.RNN(keras.layers.LSTMCell(128), return_sequences=True),
            keras.layers.TimeDistributed(keras.layers.Dense(1))
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(self.x_train, self.x_train, epochs=epochs)
        result = model.predict(self.x_train)
        return result
    
    def rnn_autoencoder(self, input_size, epochs):
        output_size = input_size
        model = keras.models.Sequential([
            keras.layers.SimpleRNN(128, input_shape=[input_size, 1]),
            keras.layers.RepeatVector(output_size),
            keras.layers.SimpleRNN(128, return_sequences=True),
            keras.layers.TimeDistributed(keras.layers.Dense(1))
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(self.x_train, self.x_train, epochs=epochs)
        result = model.predict(self.x_train)
        return result

# Load Dataset

In [ ]:
    import os
    path = os.path.join(Datasets.MFPT_BASE, Datasets.MFPT_DATA, Datasets.MFPT_MAT)
    
    mat = loadmat(path)
    keys = mat['bearing'].dtype.fields
    mat_dict = {
        list(keys.keys())[0] : mat['bearing'][0][0][0],
        list(keys.keys())[1] : mat['bearing'][0][0][1],
        list(keys.keys())[2] : mat['bearing'][0][0][2],
        list(keys.keys())[3] : mat['bearing'][0][0][3]
    }
    dataset = Preprocessing.create_dataset(mat_dict['gs'], 50)

# LSTM AutoEncoder

# Train Model & Save Result

In [ ]:
import datetime
now = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M')

epochs = 10
input_size = 50

model = Model(dataset)
result = model.lstm_autoencoder(input_size, epochs=epochs)


In [ ]:
# Save
import os

DIR = "./results"

if not os.path.exists(DIR):
    os.makedirs(DIR)
    
np.save(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}.npy', result) #Save Results to npy file




# Load Result

In [ ]:
result = np.load(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}.npy')
#print(f"{dataset}, {result}")

In [ ]:
import matplotlib.pyplot as plt

dataset = np.reshape(dataset, dataset.size)
result = np.reshape(result, result.size)

fig, axs = plt.subplots(figsize = (20,10)) # This is your answer to resize the figure
# The below will help you expand on your question and resize individual elements within your figure. Experiement with the below parameters.
axs.set_title("Data", fontsize = 17.5)
axs.tick_params(axis = 'x', labelsize = 14)
axs.set_xlabel('Timestamp', size = 15)
axs.tick_params(axis = 'y', labelsize =14)
axs.set_ylabel('Value', size = 15)

plt.plot(dataset[:300], 'r', linewidth=1)
plt.plot(result[:300], 'g')